In [1]:
!pip install requests pandas

In [11]:
import requests
import time
import pandas as pd
from scipy.stats import ttest_ind

In [ ]:
TOKEN = "TOKEN"
HEADERS = {"Authorization": f"Bearer {TOKEN}"}

In [4]:
def medir_rest(n=30):
    url = "https://api.github.com/repos/torvalds/linux"
    resultados = []
    for i in range(n):
        try:
            print(f"REST {i+1}/{n}...")
            inicio = time.time()
            r = requests.get(url, headers=HEADERS)
            fim = time.time()
            resultados.append({
                "tipo": "REST",
                "tempo_ms": (fim - inicio) * 1000,
                "tamanho_bytes": len(r.content)
            })
        except Exception as e:
            print("Erro REST:", e)
    return resultados

In [6]:
def medir_graphql(n=30):
    url = "https://api.github.com/graphql"
    query = """
    {
      repository(owner: "torvalds", name: "linux") {
        name
        description
        stargazerCount
        forkCount
        createdAt
        updatedAt
      }
    }
    """
    resultados = []
    for i in range(n):
        try:
            print(f"GraphQL {i+1}/{n}...")
            inicio = time.time()
            r = requests.post(url, json={"query": query}, headers=HEADERS)
            fim = time.time()
            resultados.append({
                "tipo": "GraphQL",
                "tempo_ms": (fim - inicio) * 1000,
                "tamanho_bytes": len(r.content)
            })
        except Exception as e:
            print("Erro GraphQL:", e)
    return resultados

In [7]:
dados_rest = medir_rest()
dados_graphql = medir_graphql()

REST 1/30...
REST 2/30...
REST 3/30...
REST 4/30...
REST 5/30...
REST 6/30...
REST 7/30...
REST 8/30...
REST 9/30...
REST 10/30...
REST 11/30...
REST 12/30...
REST 13/30...
REST 14/30...
REST 15/30...
REST 16/30...
REST 17/30...
REST 18/30...
REST 19/30...
REST 20/30...
REST 21/30...
REST 22/30...
REST 23/30...
REST 24/30...
REST 25/30...
REST 26/30...
REST 27/30...
REST 28/30...
REST 29/30...
REST 30/30...
GraphQL 1/30...
GraphQL 2/30...
GraphQL 3/30...
GraphQL 4/30...
GraphQL 5/30...
GraphQL 6/30...
GraphQL 7/30...
GraphQL 8/30...
GraphQL 9/30...
GraphQL 10/30...
GraphQL 11/30...
GraphQL 12/30...
GraphQL 13/30...
GraphQL 14/30...
GraphQL 15/30...
GraphQL 16/30...
GraphQL 17/30...
GraphQL 18/30...
GraphQL 19/30...
GraphQL 20/30...
GraphQL 21/30...
GraphQL 22/30...
GraphQL 23/30...
GraphQL 24/30...
GraphQL 25/30...
GraphQL 26/30...
GraphQL 27/30...
GraphQL 28/30...
GraphQL 29/30...
GraphQL 30/30...


In [8]:
df = pd.DataFrame(dados_rest + dados_graphql)
df.to_csv("resultados_github.csv", index=False)
df.head()

,tipo,tempo_ms,tamanho_bytes
0,REST,267.039061,5020
1,REST,251.320124,5020
2,REST,202.954054,5020
3,REST,237.681627,5020
4,REST,264.796257,5020


In [12]:
desc = df.groupby("tipo").agg({
    "tempo_ms": ["mean", "std", "min", "max"],
    "tamanho_bytes": ["mean", "std", "min", "max"]
})

desc

tempo_ms                                    tamanho_bytes       \
               mean        std         min         max          mean  std   
tipo                                                                        
GraphQL  212.206801  21.812283  186.532736  272.601366         193.0  0.0   
REST     234.645557  23.568358  202.954054  313.890457        5020.0  0.0   

                     
          min   max  
tipo                 
GraphQL   193   193  
REST     5020  5020

In [14]:
rest = df[df["tipo"] == "REST"]
graphql = df[df["tipo"] == "GraphQL"]

ttempo = ttest_ind(rest["tempo_ms"], graphql["tempo_ms"])
ttamanho = ttest_ind(rest["tamanho_bytes"], graphql["tamanho_bytes"])

print("p-valor tempo:", ttempo.pvalue)
print("p-valor tamanho:", ttamanho.pvalue)


p-valor tempo: 0.00031991043866980883
p-valor tamanho: 0.0
